In [1]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot') #style
matplotlib.rcParams['figure.figsize'] = (12, 5)#size graph
df = pd.read_hdf("./data.h5")

# print(df.shape)
# df.sample(5)

df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)
print(df_customers)

df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
df_customers.head(5)
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc
print('.....................................')
df_customers["most_revenue_customer"].value_counts(normalize=True) *100
print('................................')
print(df_customers[df_customers["most_revenue_customer"]]['customer_id'])
print('................................')
top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())
len(top_customers)
df["most_revenue_customer"] = df["customer_id"].map(lambda x: x in top_customers)
df["most_revenue_customer"].value_counts()

      customer_id  customer_price_total
448           448              61130939
907           907              53007160
2647         2647              33695110
179           179              31039769
66             66              29774259
...           ...                   ...
4594         4594                   295
180           180                   250
4286         4286                     0
4784         4784                     0
4785         4785                     0

[5879 rows x 2 columns]
.....................................
................................
448      448
907      907
2647    2647
179      179
66        66
        ... 
2142    2142
1550    1550
1584    1584
1135    1135
302      302
Name: customer_id, Length: 1324, dtype: int64
................................


True     539072
False    281834
Name: most_revenue_customer, dtype: int64

In [4]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GroupKFold

import gc

df = pd.read_hdf("./data.h5")

print(df.shape)
df.sample(5)

df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)

df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

df_customers = (
    df
    .groupby("customer_id")
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    ).reset_index()
)

df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)

df_customers

df_customers["most_revenue_customer"].value_counts(normalize=True)

(820906, 9)


,customer_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled,most_revenue_customer
0,0,159,93,22976,895827,5746671,1,0.010966,True
1,1,40,184,3514,205962,910113,1,0.097727,True
2,2,30,139,8971,95500,1337464,1,0.009050,True
3,3,9,40,1108,18626,226381,1,0.000000,False
4,4,14,11,390,20028,346978,1,0.307692,True
...,...,...,...,...,...,...,...,...,...
5874,5874,2,46,213,15654,48343,1,0.040000,False
5875,5875,1,21,261,4944,30081,1,0.000000,False
5876,5876,1,69,367,33622,110868,1,0.000000,False
5877,5877,1,33,85,7995,13458,1,0.000000,False
